# Identifying the outliers using spectral clustering

# Authors

M Anand Krishna - cs22mtech14003

K S Akash - cs22mtech11012

K Manish - cs22mtech11008

A SivaSai - cs22mtech11013

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

### Load the Dataset

First, we need to load the dataset into a pandas DataFrame. To do this, we use the `pd.read_csv()` function, which reads a CSV file and returns a DataFrame. 


In [2]:
# Load the data into a pandas DataFrame (Change the path accordingly)
data = pd.read_csv('data.csv')

#Normalize
scaler = StandardScaler()
data_norm = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

### Compute the Similarity Matrix

To perform spectral clustering, we first need to compute a similarity matrix, which captures the pairwise similarity between data points. In this case, we use a Gaussian kernel to compute the similarity matrix.

The Gaussian kernel is a function that measures the similarity between data points based on their Euclidean distance. It has a parameter `sigma`, which controls the smoothness of the similarity function. 


In [3]:
# Compute the similarity matrix using a Gaussian kernel
sigma = 1.0
similarity_matrix = pairwise_distances(data_norm, metric='euclidean')
similarity_matrix = np.exp(-similarity_matrix ** 2 / (2 * sigma ** 2))



### Compute the Laplacian Matrix
Next, we need to compute the Laplacian matrix, which is a fundamental step in spectral clustering. The Laplacian matrix is derived from the similarity matrix and helps us find the optimal partitioning of the data. To get L, subtract the similarity_matrix from the degree matrix D to obtain the Laplacian matrix L.

In [4]:

# Compute the Laplacian matrix
D = np.diag(np.sum(similarity_matrix, axis=1))
L = D - similarity_matrix

### Compute Eigenvectors and Eigenvalues

In spectral clustering, we need to compute the eigenvectors and eigenvalues of the Laplacian matrix `L`. The eigenvectors corresponding to the smallest eigenvalues provide us with a lower-dimensional representation of the data, which can be used for clustering.

To compute the eigenvectors and eigenvalues, we use NumPy's `linalg.eig` function. We then sort the eigenvalues in ascending order and rearrange the eigenvectors accordingly 


In [5]:
# Compute the eigenvectors and eigenvalues of the Laplacian matrix
eigenvalues, eigenvectors = np.linalg.eig(L)
idx = eigenvalues.argsort()
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

### K-Means Clustering

Now that we have computed the eigenvectors of the Laplacian matrix, we can use k-means clustering to partition the data points into clusters. We first select the first `k` eigenvectors, where `k` is the desired number of clusters. Then, we normalize the selected eigenvectors using L2 normalization.

Next, we use the `KMeans` class from scikit-learn to fit a k-means model to the eigenvector representation of the data. Finally, we obtain the cluster labels for each data point.
 


In [6]:
# Using k-means clustering to cluster the data points in the new eigenvector representation
k = 6
eigenvectors = eigenvectors[:, :k] 
eigenvectors = normalize(eigenvectors, axis=1, norm='l2')
kmeans_model = KMeans(n_clusters=k, n_init=10).fit(eigenvectors)  # Set n_init explicitly
labels = kmeans_model.labels_

### Cluster Centroids and Outlier Detection

After obtaining the cluster labels, we can compute the cluster centroids, which are the mean of the eigenvector representations of the data points within each cluster. We use the `cluster_centers_` attribute of the fitted `KMeans` model to get the centroids.

To identify outliers within each cluster, we first define a threshold multiplier. The threshold for each cluster is set to the mean distance of the data points within the cluster to its centroid, multiplied by the threshold multiplier.

We then iterate over the clusters and calculate the distances between the eigenvectors of the data points within the cluster and the cluster centroid. If the distance for any data point exceeds the threshold, it is flagged as an outlier.



In [7]:
# Compute the cluster centroids, handling empty clusters
centroids = kmeans_model.cluster_centers_

# Define the threshold to flag outliers
threshold_multiplier = 1.5

# Analyze the clusters to identify outliers, handling empty clusters
for label in set(labels):
    cluster = data[labels == label]
    if len(cluster) == 0:
        continue
    centroid = centroids[label]
    distances = np.linalg.norm(eigenvectors[labels == label] - centroid, axis=1)
    mean_distance = np.mean(distances)
    threshold = mean_distance * threshold_multiplier
    outliers = np.where(distances > threshold)[0]
    if len(outliers) > 0:
        print(f'Outliers detected in cluster {label}: {list(cluster.iloc[outliers].index)}')


Outliers detected in cluster 0: [3, 37, 59, 68, 82, 206, 232, 249, 251, 309, 314, 361, 408, 432, 527, 533, 689, 720, 1065]
Outliers detected in cluster 1: [608, 899]
